In [2]:
import requests
import pandas as pd
import json
import mysql.connector as mysql
from sqlalchemy import create_engine
import hashlib
from datetime import datetime


/var/folders/kk/m7gtvp5s53d8zr02s1ls5gfm0000gn/T/ipykernel_2610/373580339.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [26]:
now = datetime.now()

response = requests.get("http://universities.hipolabs.com/search?country")

text = response.json()
text = json.dumps(text);

#json text to python dict

data = json.loads(text)

data

[{'domains': ['mbzuai.ac.ae'],
  'country': 'United Arab Emirates',
  'web_pages': ['https://mbzuai.ac.ae/'],
  'name': 'Mohamed bin Zayed University of Artificial Intelligence (MBZUAI)',
  'alpha_two_code': 'AE',
  'state-province': 'Abu Dhabi'},
 {'domains': ['sempreceub.com', 'uniceub.br'],
  'country': 'Brazil',
  'web_pages': ['https://www.uniceub.br'],
  'name': 'Centro Universitário de Brasília, UNICEUB',
  'alpha_two_code': 'BR',
  'state-province': None},
 {'domains': ['student.karazin.ua', 'karazin.ua'],
  'country': 'Ukraine',
  'web_pages': ['https://karazin.ua'],
  'name': 'Kharkiv National University',
  'alpha_two_code': 'UA',
  'state-province': None},
 {'domains': ['usm.cl'],
  'country': 'Chile',
  'web_pages': ['https://usm.cl'],
  'name': 'Universidad Técnica Federico Santa María',
  'alpha_two_code': 'CL',
  'state-province': None},
 {'domains': ['ieseg.fr'],
  'country': 'France',
  'web_pages': ['https://ieseg.fr'],
  'name': 'IÉSEG School of Management',
  'alph

In [11]:

connection = mysql.connect(user='root', 
                           password='softzino',
                           host='127.0.0.1',
                           database='new_schema')
connection.is_connected()

True

In [28]:
def my_hashing(string):

    string = bytes(string, 'utf-8')
    
    sha256 = hashlib.sha256()

    sha256.update(string)
    
    string_hash = sha256.hexdigest()
     
    return string_hash

In [29]:
list_of_values = []

for val in data:
    mylist = []
    my_string = ""
    for key in val:
        mylist.append(val[key])

        fval = val[key]

        if(type(fval) == type([])):
            fval = json.dumps(fval)
            
        fval = connection._cmysql.escape_string(str(fval))
        fval = fval.decode('utf-8')

        my_string += fval
        
    #print(my_string)
    hash_value = my_hashing(my_string) 
    
    mylist.append(str(hash_value))
    
    list_of_values.append(mylist)

print(list_of_values)

[[['mbzuai.ac.ae'], 'United Arab Emirates', ['https://mbzuai.ac.ae/'], 'Mohamed bin Zayed University of Artificial Intelligence (MBZUAI)', 'AE', 'Abu Dhabi', '7600eb5538ed2a86fb2edd1c1e31fafcc80d92ea242a7416bf8ff139b78d1cfe'], [['sempreceub.com', 'uniceub.br'], 'Brazil', ['https://www.uniceub.br'], 'Centro Universitário de Brasília, UNICEUB', 'BR', None, '8245ad689ae0030778a7edd65817c46c095d1c33dd6c55a93626dabbe5580b06'], [['student.karazin.ua', 'karazin.ua'], 'Ukraine', ['https://karazin.ua'], 'Kharkiv National University', 'UA', None, '56a8be022f2d8c74c87f016466ce20ded3e4757ee17e99e308425b740590acd9'], [['usm.cl'], 'Chile', ['https://usm.cl'], 'Universidad Técnica Federico Santa María', 'CL', None, '4e69ade241ee488c503e2b74da13ed0873b2cd95c0eb90f3f8f56a7db31e79bb'], [['ieseg.fr'], 'France', ['https://ieseg.fr'], 'IÉSEG School of Management', 'FR', None, '174848b8f66f9c0415392b5d83ed41928dc6104415869f37576dd1cd0d513ec2'], [['mail2.sysu.edu.cn', 'mail.sysu.edu.cn'], 'China', ['https://

In [38]:
def update_uni(uni_name):
    
    cursor = connection.cursor()

    main_query = f"UPDATE All_uni SET updated_date ='{datetime.now()}' WHERE name = '{uni_name}'"
    print(main_query)

    cursor.execute(main_query)
    connection.commit()


In [ ]:
def update_rows(list_of_val):

    cursor = connection.cursor()
    
    for val in list_of_val:
        my_uni_name = val[3]
        my_hash = val[6]

        my_uni_name = connection._cmysql.escape_string(str(my_uni_name))
        my_uni_name = my_uni_name.decode('utf-8')

        #print(my_uni_name, my_hash)

        query_text = f"select name, hash_value from All_uni where name = '{my_uni_name}'"

        cursor.execute(query_text)
        
        query_output = cursor.fetchall()
        
        found_hash_val = query_output[0][1]

        # print(my_hash, found_hash_val)

        if found_hash_val == my_hash:
            update_uni(my_uni_name)   
        
        # break
        

update_rows(list_of_values)

7600eb5538ed2a86fb2edd1c1e31fafcc80d92ea242a7416bf8ff139b78d1cfe 7600eb5538ed2a86fb2edd1c1e31fafcc80d92ea242a7416bf8ff139b78d1cfe
UPDATE All_uni SET updated_date ='2024-02-23 16:26:32.953987' WHERE name = 'Mohamed bin Zayed University of Artificial Intelligence (MBZUAI)'
8245ad689ae0030778a7edd65817c46c095d1c33dd6c55a93626dabbe5580b06 8245ad689ae0030778a7edd65817c46c095d1c33dd6c55a93626dabbe5580b06
UPDATE All_uni SET updated_date ='2024-02-23 16:26:33.001285' WHERE name = 'Centro Universitário de Brasília, UNICEUB'
56a8be022f2d8c74c87f016466ce20ded3e4757ee17e99e308425b740590acd9 56a8be022f2d8c74c87f016466ce20ded3e4757ee17e99e308425b740590acd9
UPDATE All_uni SET updated_date ='2024-02-23 16:26:33.040782' WHERE name = 'Kharkiv National University'
4e69ade241ee488c503e2b74da13ed0873b2cd95c0eb90f3f8f56a7db31e79bb 4e69ade241ee488c503e2b74da13ed0873b2cd95c0eb90f3f8f56a7db31e79bb
UPDATE All_uni SET updated_date ='2024-02-23 16:26:33.067829' WHERE name = 'Universidad Técnica Federico Santa Mar

In [12]:
cursor = connection.cursor(dictionary=True)

In [13]:
cursor.execute('SELECT * FROM All_country')
res = cursor.fetchall()

In [14]:
res

[{'Short_notation': 'DZ', 'country': 'Algeria', 'region': 'Africa'},
 {'Short_notation': 'AO', 'country': 'Angola', 'region': 'Africa'},
 {'Short_notation': 'BJ', 'country': 'Benin', 'region': 'Africa'},
 {'Short_notation': 'BW', 'country': 'Botswana', 'region': 'Africa'},
 {'Short_notation': 'BF', 'country': 'Burkina Faso', 'region': 'Africa'},
 {'Short_notation': 'BI', 'country': 'Burundi', 'region': 'Africa'},
 {'Short_notation': 'CV', 'country': 'Cabo Verde', 'region': 'Africa'},
 {'Short_notation': 'CM', 'country': 'Cameroon', 'region': 'Africa'},
 {'Short_notation': 'CF',
  'country': 'Central African Republic (the)',
  'region': 'Africa'},
 {'Short_notation': 'TD', 'country': 'Chad', 'region': 'Africa'},
 {'Short_notation': 'KM', 'country': 'Comoros (the)', 'region': 'Africa'},
 {'Short_notation': 'CD',
  'country': 'Congo (the Democratic Republic of the)',
  'region': 'Africa'},
 {'Short_notation': 'CG', 'country': 'Congo (the)', 'region': 'Africa'},
 {'Short_notation': 'DZ', '

In [25]:
for r in res:
    my_country = r['country']
    

10
